In [ ]:
%pip install wandb

In [3]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error, matthews_corrcoef
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score



#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv





import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings


matplotlib.rc('image', cmap='Greys')

from fastkaggle import setup_comp



from openfe import OpenFE, transform
from autogluon.tabular import TabularDataset, TabularPredictor

import h2o
from h2o.automl import H2OAutoML

import gc

from xgboost import plot_importance
import wandb
from fastai.callback.wandb import *



#from IPython.display import FileLink

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
!ls

AutogluonModels		playground-series-s4e8.zip	    submission.csv
main.py			poisonous_mushrooms_classification  xgb_model.json
playground-series-s4e8	requirements.txt


In [14]:
# start logging a wandb run
wandb.init(project='s04e08_fastai_nn_gbms')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rikugan (rikugandev). Use `wandb login --relogin` to force relogin


In [15]:
# optional: mark the run as completed
#wandb.finish()

In [5]:
path = Path('playground-series-s4e8/')
path

Path('playground-series-s4e8')

In [6]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [7]:
# Make a copy of the training dataframe to avoid modifying the original
train_df_with_nan = train_df.copy()

# Add a NaN value to a random row in the 'stem-height' column
random_index = np.random.choice(train_df_with_nan.index)
train_df_with_nan.loc[random_index, 'stem-height'] = np.nan

# Verify the NaN was added
print(train_df_with_nan['stem-height'].isna().sum())

1


In [8]:
cont_names,cat_names = cont_cat_split(train_df_with_nan, dep_var='class')
len(cat_names),len(cont_names)

(17, 3)

In [9]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df_with_nan))

In [10]:
to = TabularPandas(train_df_with_nan, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='class',
                   y_block=CategoryBlock(),
                   splits=splits)

In [11]:
dls = to.dataloaders(bs=64)
#dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test_df)

In [12]:
dls

In [13]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [15]:
X_train

,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,cap-diameter_na,stem-height_na,cap-diameter,stem-height,stem-width
id,,,,,,,,,,,,,,,,,,,,,,
547711,72,60,62,9,45,0,49,0,0,56,0,0,6,19,23,30,1,1,1,-0.948561,-0.851246,-1.081126
303590,72,56,64,9,53,0,60,0,59,56,20,22,19,28,0,26,3,1,1,0.451292,0.741772,0.548564
2524055,54,82,64,9,45,32,49,0,55,47,0,0,6,19,0,31,3,1,1,-1.040883,-0.380750,-0.940273
2096058,49,77,75,9,53,29,60,0,59,56,0,0,19,18,0,26,3,1,1,0.230150,2.197716,0.090175
2055442,49,77,64,9,76,29,49,0,44,39,0,0,6,19,0,26,3,1,1,-0.538481,-0.380750,-0.899500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175917,72,57,67,9,0,29,60,0,0,56,20,22,19,20,0,26,1,1,1,0.595142,1.853180,0.869807
486908,54,65,64,9,45,29,62,0,44,47,0,0,19,0,0,26,3,1,1,-0.154166,-0.506710,-0.646212
1072035,54,73,56,9,67,0,62,0,0,47,0,0,6,19,0,26,4,1,1,1.823235,-0.732696,2.969005


In [24]:
!ls models

In [21]:
!ls /kaggle/working

models


In [20]:
!mkdir /kaggle/working/models

# Neural Network

In [26]:
learn = tabular_learner(dls, metrics=MatthewsCorrCoef())
#learn.fit_one_cycle(10)

In [21]:
dl = test_dl
nn_preds = learn.get_preds(dl=dl)
nn_preds_x = learn.get_preds()[0]
a_preds, _ = learn.get_preds(dl=dl)
nn_preds_y = a_preds.squeeze(1)
nn_preds_proba_b = (a_preds[:, 1])

In [24]:
nn_preds_x.shape,nn_preds_y.shape

(torch.Size([623389, 2]), torch.Size([2077964, 2]))

In [23]:
nn_preds_x.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [27]:
nn_preds_x

tensor([[9.7270e-01, 2.7302e-02],
        [9.9873e-01, 1.2673e-03],
        [9.9889e-01, 1.1104e-03],
        ...,
        [9.9994e-01, 5.5118e-05],
        [4.3098e-03, 9.9569e-01],
        [5.3216e-03, 9.9468e-01]])

In [43]:
final_preds_scoring = nn_preds_x[:, 1]

In [44]:
final_preds_scoring.shape

torch.Size([623389])

In [46]:
rounded_nn_scoring_preds = torch.round(final_preds_scoring).long()
rounded_nn_scoring_preds

tensor([0, 0, 0,  ..., 0, 1, 1])

In [48]:
nn_score = matthews_corrcoef(y_test,rounded_nn_scoring_preds)
nn_score

0.9838791737078751

In [49]:
final_preds = nn_preds_y[:, 1]

In [50]:
final_preds.shape

torch.Size([2077964])

In [32]:
final_preds

tensor([2.7302e-02, 1.2673e-03, 1.1104e-03,  ..., 5.5118e-05, 9.9569e-01, 9.9468e-01])

In [33]:
learn.save('mushroom_prediction_model')

Path('models/mushroom_prediction_model.pth')

In [27]:
learn.load('mushroom_prediction_nn')

FileNotFoundError: [Errno 2] No such file or directory: 'models/mushroom_prediction_nn.pth'

In [41]:
!rm submission.csv

rm: cannot remove 'submission.csv': No such file or directory


In [51]:
rounded_nn_sub_preds = torch.round(final_preds).long()
rounded_nn_sub_preds

tensor([0, 1, 1,  ..., 1, 0, 0])

In [52]:
mapping = dict(enumerate(dls.vocab))
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] = [mapping[pred.item()] for pred in rounded_nn_sub_preds]
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
print(sub)

              id class
0        3116945     e
1        3116946     p
2        3116947     p
3        3116948     p
4        3116949     e
...          ...   ...
2077959  5194904     p
2077960  5194905     p
2077961  5194906     p
2077962  5194907     e
2077963  5194908     e

[2077964 rows x 2 columns]


# XGBoost

In [14]:
params_xgb = {
              
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'booster': 'gbtree',
    'n_estimators': 5000,       
    'learning_rate': 0.55,                   
    'min_child_weight': 17,      
    'reg_lambda': 0.2,           
    'reg_alpha': 7,              
    'max_bin': 52000,            
    'colsample_bytree': 0.65,    
    'max_delta_step': 2,         
    #'random_state': 0,
    #'device' : 'cuda',
    #'devices': '0:1',
    'tree_method': 'hist'
}


In [15]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [16]:
%%time

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_leaves': trial.suggest_int('max_leaves', 0, 1000),
        #'max_bin': trial.suggest_int('max_bin', 200, 1000),
        #'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        #'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        #'tree_method': trial.suggest_categorical('tree_method', ['exact', 'approx', 'hist']),
        #'use_label_encoder': False,
        #'eval_metric': 'logloss',
        'device': 'gpu',
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 50)
    }
    
    K_FOLDS = 5  # Number of folds for cross-validation
    kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    fold_scores = []
    
    #for train_index, test_index in kfold.split(train_df):
    #print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

    for train_index, val_index in kfold.split(X_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold
        
        xgb_model_fold = xgb.XGBClassifier(**params)
        xgb_model_fold.fit(X_fold_train, y_fold_train)
        
        # Predict probabilities on the validation set for the current fold
        y_pred_fold = xgb_model_fold.predict(X_fold_val)
        y_pred_fold_tt = xgb_model_fold.predict(test_dl.xs)
        
        # Calculate and store the AUC-ROC score for the current fold
        score = matthews_corrcoef(y_fold_val, y_pred_fold)
        fold_scores.append(score)
        
        # Calculate the average score across all folds
        return np.mean(fold_scores)

# Create and run the study
study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-08-03 12:56:25,626] A new study created in memory with name: no-name-a4ae0304-e79e-4715-8dc6-2439b51913ec
[I 2024-08-03 12:56:57,461] Trial 0 finished with value: 0.9836513599373877 and parameters: {'num_leaves': 320, 'learning_rate': 0.07257005721594278, 'n_estimators': 1225, 'subsample_for_bin': 172567, 'min_child_samples': 223, 'reg_alpha': 0.006502000785097662, 'reg_lambda': 8.675307976899775e-05, 'colsample_bytree': 0.9350638004692479, 'subsample': 0.9727470703757719, 'max_depth': 10, 'min_child_weight': 16, 'gamma': 0.0001702783205227671, 'max_leaves': 568}. Best is trial 0 with value: 0.9836513599373877.
[I 2024-08-03 12:57:09,306] Trial 1 finished with value: 0.0 and parameters: {'num_leaves': 471, 'learning_rate': 0.00019237305096546508, 'n_estimators': 219, 'subsam

Best value: 0.984563139728494
Best trial: {'num_leaves': 375, 'learning_rate': 0.0071851097672265855, 'n_estimators': 1851, 'subsample_for_bin': 104472, 'min_child_samples': 454, 'reg_alpha': 0.0011093389440775324, 'reg_lambda': 0.15936937364526085, 'colsample_bytree': 0.4533510196891779, 'subsample': 0.9272807625455266, 'max_depth': 20, 'min_child_weight': 4, 'gamma': 0.0033558210410357075, 'max_leaves': 873}
CPU times: user 1h 20min 26s, sys: 1min 10s, total: 1h 21min 36s
Wall time: 1h 16min 20s


In [ ]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef
from optuna.samplers import TPESampler

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 50)
    }
    
    K_FOLDS = 5  # Number of folds for cross-validation
    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    fold_scores = []

    for train_index, val_index in skf.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold
        xgb_model_fold = xgb.XGBClassifier(**params)
        xgb_model_fold.fit(X_fold_train, y_fold_train)
        
        # Predict on the validation set for the current fold
        y_pred_fold = xgb_model_fold.predict(X_fold_val)
        
        # Calculate and store the Matthews Correlation Coefficient for the current fold
        score = matthews_corrcoef(y_fold_val, y_pred_fold)
        fold_scores.append(score)
        
        print(f"Fold {fold} MCC: {score}")

    # Return the average score across all folds
    mean_score = np.mean(fold_scores)
    print(f"Mean MCC: {mean_score}")
    return mean_score

# Create and run the study
study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

In [19]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

Best value: 0.984563139728494
Best trial: {'num_leaves': 375, 'learning_rate': 0.0071851097672265855, 'n_estimators': 1851, 'subsample_for_bin': 104472, 'min_child_samples': 454, 'reg_alpha': 0.0011093389440775324, 'reg_lambda': 0.15936937364526085, 'colsample_bytree': 0.4533510196891779, 'subsample': 0.9272807625455266, 'max_depth': 20, 'min_child_weight': 4, 'gamma': 0.0033558210410357075, 'max_leaves': 873}


In [ ]:
best_params = {
    'num_leaves': 375, 'learning_rate': 0.0071851097672265855, 'n_estimators': 1851, 'subsample_for_bin': 104472, 'min_child_samples': 454, 'reg_alpha': 0.0011093389440775324, 'reg_lambda': 0.15936937364526085, 'colsample_bytree': 0.4533510196891779, 'subsample': 0.9272807625455266, 'max_depth': 20, 'min_child_weight': 4, 'gamma': 0.0033558210410357075, 'max_leaves': 873}

 parameters: {'num_leaves': 375, 'learning_rate': 0.0071851097672265855, 'n_estimators': 1851, 'subsample_for_bin': 104472, 'min_child_samples': 454, 'reg_alpha': 0.0011093389440775324, 'reg_lambda': 0.15936937364526085, 'colsample_bytree': 0.4533510196891779, 'subsample': 0.9272807625455266, 'max_depth': 20, 'min_child_weight': 4, 'gamma': 0.0033558210410357075, 'max_leaves': 873}.

In [23]:
optuna_params = {
              
    'num_leaves': 375, 
    'learning_rate': 0.0071851097672265855, 
    'n_estimators': 1851, 
    'subsample_for_bin': 104472,
    'min_child_samples': 454,
    'reg_alpha': 0.0011093389440775324,
    'reg_lambda': 0.15936937364526085,
    'colsample_bytree': 0.4533510196891779,
    'subsample': 0.9272807625455266,
    'max_depth': 20,
    'min_child_weight': 4,
    'gamma': 0.0033558210410357075,
    'max_leaves': 873,
    'device': 'gpu'
}


In [24]:
%%time
xgb_model = xgb.XGBClassifier(**optuna_params)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

xgb_score = matthews_corrcoef(y_test,xgb_preds_x)
xgb_score

CPU times: user 1min 48s, sys: 1.15 s, total: 1min 49s
Wall time: 1min 45s


0.9847493710839511

In [27]:
!ls

submission.csv


In [26]:
mapping = dict(enumerate(dls.vocab))
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] = [mapping[pred.item()] for pred in xgb_preds]
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
print(sub)

              id class
0        3116945     e
1        3116946     p
2        3116947     p
3        3116948     p
4        3116949     e
...          ...   ...
2077959  5194904     p
2077960  5194905     p
2077961  5194906     p
2077962  5194907     e
2077963  5194908     e

[2077964 rows x 2 columns]


# LGBM

In [ ]:
%%time

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'device_type': 'gpu'
    }
    
    K_FOLDS = 5  # Number of folds for cross-validation
    kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    fold_scores = []
    
    #for train_index, test_index in kfold.split(train_df):
    #print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

    for train_index, val_index in kfold.split(X_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold
        lgb_model_fold = lgb.LGBMClassifier(**params)  # Use params, not lgbm_params
        lgb_model_fold.fit(X_fold_train, y_fold_train)
        
        y_pred_fold = lgb_model_fold.predict(X_fold_val)
        y_pred_fold_tt = lgb_model_fold.predict(test_dl.xs)
        
        # Calculate and store the AUC-ROC score for the current fold
        score = matthews_corrcoef(y_fold_val, y_pred_fold)
        fold_scores.append(score)
        
        # Calculate the average score across all folds
        return np.mean(fold_scores)

# Create and run the study
study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-08-03 14:52:46,693] A new study created in memory with name: no-name-e3b3fe21-53bf-41c8-b662-b23f7adbeb30


[LightGBM] [Info] Number of positive: 1090782, number of negative: 904062
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 966
[LightGBM] [Info] Number of data points in the train set: 1994844, number of used features: 20
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (22.83 MB) transferred to GPU in 0.039999 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546801 -> initscore=0.187752
[LightGBM] [Info] Start training from score 0.187752
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2024-08-03 14:57:18,510] Trial 0 finished with value: 0.9836442546936054 and parameters: {'num_leaves': 320, 'learning_rate': 0.13981961408994045, 'n_estimators': 843, 'subsample_for_bin': 172567, 'min_child_samples': 223, 'reg_alpha': 0.0028770084050677926, 'reg_lambda': 2.3761404778025532e-05, 'colsample_bytree': 0.9350638004692479, 'subsample': 0.9727470703757719, 'max_depth': 6}. Best is trial 0 with value: 0.9836442546936054.


...

[LightGBM] [Info] Number of positive: 1090782, number of negative: 904062
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 1994844, number of used features: 20
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (22.83 MB) transferred to GPU in 0.025495 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546801 -> initscore=0.187752
[LightGBM] [Info] Start training from score 0.187752
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

[I 2024-08-03 16:08:33,411] Trial 19 finished with value: 0.9592259112045235 and parameters: {'num_leaves': 259, 'learning_rate': 0.004261193412144711, 'n_estimators': 467, 'subsample_for_bin': 284425, 'min_child_samples': 375, 'reg_alpha': 8.027643657785602e-05, 'reg_lambda': 1.8799499177898148e-07, 'colsample_bytree': 0.5526138890622357, 'subsample': 0.2935218702429067, 'max_depth': 7}. Best is trial 8 with value: 0.9837296376074599.


[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Number of positive: 1090782, number of negative: 904062
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 923
[LightGBM] [Info] Number of data points in the train set: 1994844, number of used features: 20
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (22.83 MB) transferred to GPU in 0.025338 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546801 -> initscore=0.187752
[LightGBM] [Info] Start training from score 0.187752


[I 2024-08-03 16:11:30,122] Trial 20 finished with value: 0.9838453525505297 and parameters: {'num_leaves': 225, 'learning_rate': 0.12275253409866282, 'n_estimators': 640, 'subsample_for_bin': 70289, 'min_child_samples': 31, 'reg_alpha': 4.704314136881364e-09, 'reg_lambda': 0.006221967452909479, 'colsample_bytree': 0.6722181067336273, 'subsample': 0.6524344083315416, 'max_depth': 14}. Best is trial 20 with value: 0.9838453525505297.


[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Number of positive: 1090782, number of negative: 904062
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 933
[LightGBM] [Info] Number of data points in the train set: 1994844, number of used features: 20
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (22.83 MB) transferred to GPU in 0.029014 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546801 -> initscore=0.187752
[LightGBM] [Info] Start training from score 0.187752


In [ ]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

In [17]:
%%time
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'device_type': 'gpu'
    }
    
    K_FOLDS = 5  # Number of folds for cross-validation
    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
    fold_scores = []
    
    for train_index, val_index in skf.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold
        lgb_model_fold = lgb.LGBMClassifier(**params)
        lgb_model_fold.fit(X_fold_train, y_fold_train)
        
        y_pred_fold = lgb_model_fold.predict(X_fold_val)
        
        # Calculate and store the MCC score for the current fold
        score = matthews_corrcoef(y_fold_val, y_pred_fold)
        fold_scores.append(score)
    
    # Calculate the average score across all folds
    return np.mean(fold_scores)

# Create and run the study
study = optuna.create_study(sampler=optuna.samplers.TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-08-04 04:25:47,945] A new study created in memory with name: no-name-d5b62d16-5be3-4a37-8772-73c5015fc046


  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 1091628, number of negative: 903216
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 963
[LightGBM] [Info] Number of data points in the train set: 1994844, number of used features: 20
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (22.83 MB) transferred to GPU in 0.032852 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547225 -> initscore=0.189464
[LightGBM] [Info] Start training from score 0.189464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

KeyboardInterrupt: 

In [18]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

Best value: 0.983809670430194
Best trial: {'num_leaves': 320, 'learning_rate': 0.13981961408994045, 'n_estimators': 843, 'subsample_for_bin': 172567, 'min_child_samples': 223, 'reg_alpha': 0.0028770084050677926, 'reg_lambda': 2.3761404778025532e-05, 'colsample_bytree': 0.9350638004692479, 'subsample': 0.9727470703757719, 'max_depth': 6}


In [23]:
%%time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef
import numpy as np
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback
from tqdm import tqdm

def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'device_type': 'gpu'
    }
    
    K_FOLDS = 5  # Number of folds for cross-validation
    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
    fold_scores = []
    
    for train_index, val_index in skf.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold with early stopping
        lgb_model_fold = lgb.LGBMClassifier(**params)
        lgb_model_fold.fit(
            X_fold_train, y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            callbacks=[
                #LightGBMPruningCallback(trial, 'binary_logloss'),
                lgb.early_stopping(stopping_rounds=50, verbose=False)
            ]
        )
        
        y_pred_fold = lgb_model_fold.predict(X_fold_val)
        #y_pred_fold_tt = lgb_model_fold.predict(test_dl.xs)
        
        # Calculate and store the MCC score for the current fold
        score = matthews_corrcoef(y_fold_val, y_pred_fold)
        fold_scores.append(score)
    
    # Calculate the average score across all folds
    return np.mean(fold_scores)

# Create and run the study with a progress bar
study = optuna.create_study(sampler=optuna.samplers.TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-08-04 05:43:37,688] A new study created in memory with name: no-name-ccdfcd01-da14-4f9e-b8f8-120ef51e4173


  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 1091628, number of negative: 903216
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 963
[LightGBM] [Info] Number of data points in the train set: 1994844, number of used features: 20
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (22.83 MB) transferred to GPU in 0.032911 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547225 -> initscore=0.189464
[LightGBM] [Info] Start training from score 0.189464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

KeyboardInterrupt: 

In [20]:
%pip install optuna-integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 2.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
best_params = study.best_trial.params
best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
final_score = matthews_corrcoef(y_test, y_pred)
print(f"Final MCC on test set: {final_score}")